# Waterbomb shell folder and designer

In [ ]:
%matplotlib widget

In [ ]:
import sympy as sp
sp.init_printing()
import numpy as np
import matplotlib.pylab as plt
import bmcs_utils.api as bu

In [ ]:
from bmcs_shell.api import WBCell4Param as WBCell
from bmcs_shell.api import WBTessellation4P

In [ ]:
gamma_ = 0.2
eta = 0.5
zeta = 0.90
b = 1200
a = b * eta
c = a * zeta
data = dict(alpha=np.pi/2 -gamma_, a=a, b=b, c=c, n_phi_plus=10, n_x_plus=10)
#data = dict(alpha=np.pi/2 - gamma_, a=500, b=1200, c=550, n_phi_plus=2, n_x_plus=2)

In [ ]:
zeta, eta,

In [ ]:
wb = WBCell(**data)
ws = WBTessellation4P(wb_cell=wb, show_wireframe=True, **data)

In [ ]:
app = bu.AppWindow(ws)
app.interact()

In [ ]:
I_m = ws.I_CDij[-1,:,-1,:]
I_m

In [ ]:
_, idx_remap = ws.unique_node_map
J_m = idx_remap[I_m]

In [ ]:
ws.X_Ia[J_m.flatten()].T

# Node map to allow for hierarchical sliding

In [ ]:
n_cells, n_ic, n_id, _, x_cell_idx, _, y_cell_idx = ws.cell_map
x_cell_idx, y_cell_idx

In [ ]:
x_idx, y_idx = x_cell_idx / 2, y_cell_idx / 2
x_idx, y_idx

In [ ]:
n_x_, n_y_ = len(x_idx), len(y_idx) 

In [ ]:
I_cell_offset = (n_ic + np.arange(n_x_*n_y_).reshape(n_x_, n_y_)) * ws.wb_cell.n_I
I_cell_offset

In [ ]:
ws.wb_cell.I_boundary[np.newaxis,np.newaxis,:,:]

In [ ]:
I_CDij_map = I_cell_offset.T[:,:,np.newaxis, np.newaxis] + ws.wb_cell.I_boundary[np.newaxis,np.newaxis,:,:]

In [ ]:
I_CDij_map[-1,:,-1,:]

# Find duplicate nodes and merge them to a single one

In [ ]:
X_Ia = np.array([[2,1,0],
                 [3,8,9],
                 [0,0,0],
                 [5,3,9],
                 [1,0,1],
                 [0,0,0],
                 [1,0,1],
                 [0,0,0]],dtype=np.float_)

In [ ]:
x_0[np.newaxis, :, :]

In [ ]:
# reshape the coordinates in array of segments to the shape (n_N, n_D
x_0 = X_Ia

In [ ]:
# construct distance vectors between every pair of nodes
x_x_0 = x_0[:, np.newaxis, :] - x_0[np.newaxis, :, :]
x_x_0

In [ ]:
# calculate the distance between every pair of nodes
dist_0 = np.sqrt(np.einsum('...i,...i', x_x_0, x_x_0))
dist_0

In [ ]:
# identify those at the same location
zero_dist = dist_0 < ws.node_match_threshold
zero_dist

In [ ]:
# get their indices
i_idx, j_idx = np.where(zero_dist)

In [ ]:
# take only the upper triangle indices
upper_triangle = i_idx < j_idx

In [ ]:
idx_multi, idx_delete = i_idx[upper_triangle], j_idx[upper_triangle]

In [ ]:
# construct a boolean array with True at valid and False at deleted
# indices
idx_unique = np.ones((len(x_0),), dtype='bool')
idx_unique[idx_delete] = False

In [ ]:
idx_multi, idx_delete

In [ ]:
idx_keep = np.ones((len(x_0),), dtype=np.bool_)
idx_keep[idx_delete] = False

In [ ]:
idx_delete, idx_multi

In [ ]:
ij_map = np.ones_like(dist_0, dtype=np.int_) + len(x_0)
ij_map

In [ ]:
i_ = np.arange(len(x_0))
idx_row = i_[idx_keep]

In [ ]:
ij_map[idx_keep,idx_keep] = np.arange(len(idx_row))
ij_map

In [ ]:
ij_map[i_idx, j_idx] = ij_map[i_idx, i_idx]
ij_map

In [ ]:
idx_remap = np.min(ij_map,axis=0)
idx_remap

## Sketch of the applied generation method / concept

Sketch the discretization into the cells the cell midpoints 
cover a cylindric surface as a structured grid. 

Midpoints are arranged at the surface of the derived cylinder
parameters which depend on the folding angle $\alpha$ and $a, b, c$
parameters of the waterbomd cell. Then, a reference cell 
in an intermediate state of folding is broadcasted 
to the prepared grid of midpoints on the cylinders. 

In [ ]:
n_phi_plus = 2
delta_phi = 0.3
phi_range = np.arange(-(n_phi_plus-1), n_phi_plus) * delta_phi
r_0 = -10

In [ ]:
phi_range

In [ ]:
X_phi_range = np.array([np.fabs(r_0) * np.sin(phi_range),
                 np.fabs(r_0) * np.cos(phi_range) + r_0]).T

In [ ]:
fix, ax = plt.subplots(1,1)
ax.plot(*X_phi_range.T);

In [ ]:
n_x_plus = 2
delta_x = 0.5
X_x_range = np.arange(-(n_x_plus-1), n_x_plus) * delta_x
X_x_range

In [ ]:
X_x_range[:, np.newaxis, np.newaxis]

In [ ]:
X_phi_range[np.newaxis,:,:]

In [ ]:
n_idx_x = len(X_x_range)
n_idx_phi = len(X_phi_range)
idx_x = np.arange(n_idx_x)
idx_phi = np.arange(n_idx_phi)
idx_phi

In [ ]:
1+1 % 2

In [ ]:
idx_x_ic = idx_x[(n_idx_x)%2::2]
idx_x_id = idx_x[(n_idx_x+1)%2::2]
idx_phi_ic = idx_phi[(n_idx_phi)%2::2]
idx_phi_id = idx_phi[(n_idx_phi+1)%2::2]

idx_x_ic, idx_phi_ic, idx_x_id, idx_phi_id

In [ ]:
idx_map_ic = np.meshgrid(idx_x_ic, idx_phi_ic)
idx_map_id = np.meshgrid(idx_x_id, idx_phi_id)

In [ ]:
idx_map_ic

In [ ]:
X_C_a = np.c_[ X_x_range[idx_map_ic[0]].reshape(-1,1), X_phi_range[idx_map_ic[1]].reshape(-1,2) ]
X_D_a = np.c_[ X_x_range[idx_map_id[0]].reshape(-1,1), X_phi_range[idx_map_id[1]].reshape(-1,2) ]

In [ ]:
X_C_a

In [ ]:
X_D_a